In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/Cost of living - purchasing power index 2024.csv')
df

In [ ]:
df = df.drop(columns = ['Rank', 'Ø Monthly income (USD)'])
df

In [ ]:
df.rename(columns={'Country/Region': 'Country Name'}, inplace=True)
df

In [ ]:
# Remove special characters from country names
df['Country Name'] = df['Country Name'].str.replace('*', '')
df['Country Name'] = df['Country Name'].str.strip()
df

In [ ]:
# ADD TO EVERY ROW OF FINAL FILE NO MATTER THE YEAR

In [ ]:
df

In [ ]:
check = df[df['iso_alpha3'].isna()][['Country Name']]
check

### Normalization Step: Add ISO-alpha3 Code

In [ ]:
try:
    unsd_df = pd.read_csv('data/UNSD — Methodology.csv', sep=';')
except FileNotFoundError:
    unsd_df = pd.read_csv('../data/UNSD — Methodology.csv', sep=';')

# Create mappings
unsd_df['Country or Area'] = unsd_df['Country or Area'].str.strip()
name_to_iso3 = unsd_df.set_index('Country or Area')['ISO-alpha3 Code'].to_dict()

m49_to_iso3 = {}
for idx, row in unsd_df.iterrows():
    try:
        m49_code = int(row['M49 Code'])
        m49_to_iso3[m49_code] = row['ISO-alpha3 Code']
    except (ValueError, TypeError):
        continue

# Apply mapping
if 'Country Name' in df.columns:
    df['iso_alpha3'] = df['Country Name'].str.strip().map(name_to_iso3)
else:
    print("  Source column 'Country/Region' not found.")

# Check results
missing_iso = df['iso_alpha3'].isna().sum()
if missing_iso > 0:
    print(f"  {missing_iso} rows have missing ISO-alpha3 codes")
    print(df[df['iso_alpha3'].isna()][['Country Name']])